# QIIME2 2019.7 MOVING PICTURES TUTORIAL

This notebook contains data and analyses from the moving pictures tutorial (https://docs.qiime2.org/2019.7/tutorials/moving-pictures/)

It also demonstrates how to use the QIIME2 jupyter in-line visualization feature



In [1]:
#import the qiime2 python module to allow for inline visualizations

import qiime2 as q2

ModuleNotFoundError: No module named 'qiime2'

In [ ]:
#which version of qiime2 and python are we using?

!qiime info

## A note about using Linux commands in Jupyter Notebooks

Since we are running this notebook with Python, it won't automatically recognize linux commands (ls, cd, mkdir, etc) or any commands from software based on linux (like qiime2 commands).

A way to get around this is to use "jupyter magic commands". The only one we'll use today is to put an exclamation mark (!) before all of our linux or qiime2 commands (aside from the python module, which we have imported as q2). This command tells jupyter that the command following it will be something other than Python.

# Obtaining and importing data

In [ ]:
#Make a directory for the tutorial data and download from the qiime2 website

!mkdir emp-single-end-sequences

!wget \
    -O "sample-metadata.tsv" \
    "https://data.qiime2.org/2019.7/tutorials/moving-pictures/sample_metadata.tsv"

!wget \
    -O "emp-single-end-sequences/sequences.fastq.gz" \
    "https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"

!wget \
    -O "emp-single-end-sequences/barcodes.fastq.gz" \
    "https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"

In [ ]:
#Import the fastq files into a QIIME2 artifact 

!qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza

## A note about fastq files

There are a few different format of fastq files your sequencer may return to you. The type of fastq you import into qiime in the above step is delimited using the "--type" flag. More information about the type of fastq's and which argument to use can be found here: https://docs.qiime2.org/2019.7/tutorials/importing/


## A note about QIIME2 file types

QIIME2 uses two file types: .qza (QIIME2 artifact) and .qzv (QIIME2 visualization). Any .qzv file can be visualized using qiime2 view (https://view.qiime2.org/) or using the jupyter in-line api (this is the method we'll be using in this notebook).

# Demultiplexing sequences

In [ ]:
# Demutliplex your sequences

!qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences demux.qza \
  --output-dir demux

In [ ]:
## change your .qza file into a .qzv file so you can view your QC stats on your demutliplexing 

!qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

In [ ]:
## Use the q2 module to view your .qzv file

q2.Visualization.load('demux.qzv')

## Use the information above to determine your truncuation position and your trim length for the next step 

#### From the tutorial website: "In the demux.qzv quality plots, we see that the quality of the initial bases seems to be high, so we won’t trim any bases from the beginning of the sequences. The quality seems to drop off around position 120, so we’ll truncate our sequences at 120 bases. This next command may take up to 10 minutes to run, and is the slowest step in this tutorial."



(see https://docs.qiime2.org/2019.7/tutorials/moving-pictures/ for more details)

# Sequence quality control and feature table construction¶


In [ ]:
#Generate ESVs (Exact Sequence Variants). QIIME2 has two packages available for this. We are using dada2 for our single end reads.
# NOTE: if you are wanting to do paired end reads, you'll use qiime dada2 denoise-paired (https://docs.qiime2.org/2019.7/plugins/available/dada2/denoise-paired/)
# NOTE: if you are wanting to join reads you'll have to use Deblur


!qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 120 \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza

In [ ]:
# change the .qza denoising stats file to a .qzv file

!qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv

#view the output
q2.Visualization.load('stats-dada2.qzv')

In [ ]:
#Change the name of some of the files to make downstream commands easier

!mv rep-seqs-dada2.qza rep-seqs.qza &&  mv table-dada2.qza table.qza

In [ ]:
# The feature-table summarize command will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics. 
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

#View the .qzv
q2.Visualization.load('table.qzv')

In [ ]:
#  The feature-table tabulate-seqs command will provide a mapping of feature IDs to sequences, and provide links to easily BLAST each sequence against the NCBI nt database.

!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

q2.Visualization.load('rep-seqs.qzv')

# Diversity analyses

In [ ]:
# Generate a phylogenetic tree to complete alpha and beta diversity analyses

!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

In [ ]:
# the core-metrics-phylogenetic method, which rarefies a FeatureTable[Frequency] to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics
# Sampling depth can be determined using output table.qzv


!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 1109 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

In [ ]:
#View unweighted unifrac beta diversity on a PCoA emperor plot

q2.Visualization.load('core-metrics-results/weighted_unifrac_emperor.qzv')

In [ ]:
#Using group significance can show you how different factors in your metadata affect your diversity

!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

q2.Visualization.load('core-metrics-results/faith-pd-group-significance.qzv')

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

q2.Visualization.load('core-metrics-results/evenness-group-significance.qzv')

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column body-site \
  --o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

q2.Visualization.load('core-metrics-results/unweighted-unifrac-body-site-significance.qzv')

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

q2.Visualization.load('core-metrics-results/unweighted-unifrac-subject-group-significance.qzv')

In [ ]:
# putting cusom axes on your PCoA emperor plot (shown above)

!qiime emperor plot \
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-DaysSinceExperimentStart.qzv

q2.Visualization.load('core-metrics-results/unweighted-unifrac-emperor-DaysSinceExperimentStart.qzv')

In [ ]:
!qiime emperor plot \
  --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/bray-curtis-emperor-DaysSinceExperimentStart.qzv

q2.Visualization.load('core-metrics-results/bray-curtis-emperor-DaysSinceExperimentStart.qzv')

In [ ]:
# Alpha rarefaction
# This visualizer computes one or more alpha diversity metrics at multiple sampling depths, in steps between 1 (optionally controlled with --p-min-depth) and the value provided as --p-max-depth

!qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 4000 \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization alpha-rarefaction.qzv

q2.Visualization.load('alpha-rarefaction.qzv')

# Taxonomic Analysis

In [ ]:
#Download machine learning classifier from the QIIME2 website: https://docs.qiime2.org/2019.7/data-resources/
#Note: This classifier was built against the greengenes database for 16S V4 sequences, QIIME2 also has one built against the SILVA database

!wget \
    -O "gg-13-8-99-515-806-nb-classifier.qza" \
    "https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza"

In [ ]:
#Use machine learning to identify your representative sequences against an external database
#This step may take a few minutes

!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

In [ ]:
#View the taxonomic designations

!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

q2.Visualization.load('taxonomy.qzv')

In [ ]:
#View the taxonomic composition of each sample in an interactive bar plot

!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

q2.Visualization.load('taxa-bar-plots.qzv')

# Differential abundance testing with ANCOM

In [ ]:
# Filter your feature table to contain only gut samples
# Filtering Data tutorial: https://docs.qiime2.org/2019.7/tutorials/filtering/

!qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "[body-site]='gut'" \
  --o-filtered-table gut-table.qza

In [ ]:
# ANCOM operates on a FeatureTable[Composition] QIIME 2 artifact, which is based on frequencies of features on a per-sample basis, but cannot tolerate frequencies of zero. 
# To build the composition artifact, a FeatureTable[Frequency] artifact must be provided to add-pseudocount (an imputation method), which will produce the FeatureTable[Composition] artifact.

!qiime composition add-pseudocount \
  --i-table gut-table.qza \
  --o-composition-table comp-gut-table.qza

In [ ]:
# run ANCOM on the subject column to determine what features differ in abundance across the gut samples of the two subjects.

!qiime composition ancom \
  --i-table comp-gut-table.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization ancom-Subject.qzv

q2.Visualization.load('ancom-Subject.qzv')

In [ ]:
#To look at differential abundance at a different taxonomic level

!qiime taxa collapse \
  --i-table gut-table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table gut-table-l6.qza

In [ ]:
# Redo the add-pseudocount on the new table

!qiime composition add-pseudocount \
  --i-table gut-table-l6.qza \
  --o-composition-table comp-gut-table-l6.qza

In [ ]:
#Run the analysis on the collapsed taxonomy

!qiime composition ancom \
  --i-table comp-gut-table-l6.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization l6-ancom-Subject.qzv

q2.Visualization.load('l6-ancom-Subject.qzv')